# This will load play data into a Kafka instance. Only run once as otherwise the topics won't be ordered.

To start Kafka 9 in Docker (using image from https://hub.docker.com/r/flozano/kafka/) run the following in the Docker bash:

```
docker run -p 2181:2181 -p 9092:9092 --env _KAFKA_advertised_host_name=`docker-machine ip \`docker-machine active\`` --env _KAFKA_advertised_port=9092 --name=local_kafka flozano/kafka &
```

To kill it run  
```
docker kill local_kafka
```
then 
```
docker rm local_kafka
```

In [1]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../src')
sys.path.append('../../msq-domain/src')

import aiostreams.operators as op
from aiostreams.config import QCConfigProvider

#QCConfigProvider().kafka_broker = '0.0.0.0:9092'
kafka_host = QCConfigProvider().kafka_broker
# !set _KAFKA_advertised_host_name=0.0.0.0
# !set _KAFKA_advertised_port=9092
print(kafka_host)

kafka:9092


In [2]:
# try writing just one record to Kafka using the basic kafka library, 
# and reading back from it, to make sure Kafka is up, 
from kafka import KafkaProducer, KafkaConsumer
import pickle

topic='test_topic_2'

kprod=KafkaProducer(bootstrap_servers=kafka_host)
kprod.send(topic,pickle.dumps('test'))

consumer = KafkaConsumer(topic,bootstrap_servers=kafka_host,auto_offset_reset='earliest',
                         group_id=None, value_deserializer=lambda x: pickle.loads(x))
next(consumer).value

'test'

In [11]:
# let's check the dump was successful
from aiostreams import AsyncKafkaSource, run
import aiostreams.operators as op
from mosaicsmartdata.common.json_convertor import *
import json

input_graph = AsyncKafkaSource('bond-quotes-topic', bootstrap_servers=kafka_host ,\
                               value_deserializer =  lambda x: x.decode('utf-8'))|\
                           op.map(lambda x: json_to_domain(x.value)) > print #op.map(lambda x: x.value ) > print
                           #op.map(lambda x: json_to_domain(x.value)) > print # op.map(lambda x: json.loads(x.value) ) > print# 
                           #op.map(lambda x: json_to_domain(x.value).timestamp) > print # op.map(lambda x: json.loads(x.value) ) > print# 
                #
run(input_graph)

<class 'mosaicsmartdata.core.quote.Quote'> :  ask = 92.71875, timestamp = 2017-09-05 03:30:01.933000, units = None, bid = 92.71875, instrument = None, duration = None, sym = 912796KN8
<class 'mosaicsmartdata.core.quote.Quote'> :  ask = 92.73438, timestamp = 2017-09-05 04:30:01.933000, units = None, bid = 92.73438, instrument = None, duration = None, sym = 912796KN8
<class 'mosaicsmartdata.core.quote.Quote'> :  ask = 92.73438, timestamp = 2017-09-05 05:00:01.933000, units = None, bid = 92.73438, instrument = None, duration = None, sym = 912796KN8
<class 'mosaicsmartdata.core.quote.Quote'> :  ask = 92.71875, timestamp = 2017-09-05 05:25:01.933000, units = None, bid = 92.71875, instrument = None, duration = None, sym = 912796KN8
<class 'mosaicsmartdata.core.quote.Quote'> :  ask = 99.716, timestamp = 2017-09-05 05:29:31.933000, units = None, bid = 99.716, instrument = None, duration = None, sym = 912796KN8
<class 'mosaicsmartdata.core.quote.Quote'> :  ask = 92.76563, timestamp = 2017-09-05

In [3]:
# let's check the dump was successful
from aiostreams import AsyncKafkaSource, run
import aiostreams.operators as op
from mosaicsmartdata.common.json_convertor import *
import json

input_graph = AsyncKafkaSource('bond-quotes-topic', bootstrap_servers=kafka_host, value_deserializer =  lambda x: x.decode('utf-8')) >print
                               #op.map(lambda x: print(x.value) > print# #op.map(lambda x: json.loads(x.value) ) > []                            
    #op.map(lambda x: json_to_domain(x.value)) > print# #op.map(lambda x: json.loads(x.value) ) > []
                           #
                            #
                #
                
#run(input_graph)
# print(input_graph.sink[0].value)
# json_to_domain(input_graph.sink[0].value)

In [5]:
# let's check the dump was successful
from aiostreams import AsyncKafkaSource, run
import aiostreams.operators as op
from mosaicsmartdata.common.json_convertor import *
import json

input_graph = AsyncKafkaSource('output-topic', bootstrap_servers=kafka_host,\
                               value_deserializer =  lambda x: x.decode('utf-8'))|\
                op.map(lambda x: x.value ) > print
                #op.map(lambda x: json_to_domain(x.value)) > print
                #op.map(lambda x: json.loads(x.value) ) > print
                
run(input_graph)

{"negotiationId": "0", "instrumentId": "912796KN8", "legMarkout": [{"markoutPrice": [{"priceType": "HEDGED_INITIAL_EDGE", "value": NaN}, {"priceType": "HEDGED_SPREAD_PRICE", "value": NaN}], "evaluatedPricingSource": "INTERNAL", "markoutPeriod": {"timeUnit": "SECONDS", "value": "-900"}}], "dateLocalDate": "2017-09-05", "venueName": "TRADEWEBUST_ASIA", "cashflowDirection": "+"}
{"negotiationId": "0", "instrumentId": "912796KN8", "legMarkout": [{"markoutPrice": [{"priceType": "HEDGED_INITIAL_EDGE", "value": NaN}, {"priceType": "HEDGED_SPREAD_PRICE", "value": NaN}], "evaluatedPricingSource": "INTERNAL", "markoutPeriod": {"timeUnit": "SECONDS", "value": "-60"}}], "dateLocalDate": "2017-09-05", "venueName": "TRADEWEBUST_ASIA", "cashflowDirection": "+"}
{"negotiationId": "1", "instrumentId": "912796KN8", "legMarkout": [{"markoutPrice": [{"priceType": "HEDGED_INITIAL_EDGE", "value": NaN}, {"priceType": "HEDGED_SPREAD_PRICE", "value": NaN}], "evaluatedPricingSource": "INTERNAL", "markoutPeriod"

In [ ]:
# now let's feed that data into our nice Kafka wrapper: first eur quotes
from aiostreams.kafka import AsyncKafkaPublisher

print(len(eurquotes))

pub1 = AsyncKafkaPublisher(bootstrap_servers=kafka_host, topic='eurquotes', value_serializer = 'json')
#pub1.verbose=True
to_async_iterable(eurquotes) > pub1


In [ ]:
# and then gbp quotes
print(len(eurgbpquotes))

pub1.topic='eurgbpquotes'
to_async_iterable(eurgbpquotes) > pub1

In [ ]:
# and now let's publish trades to a joint topic, just for the fun of it

eurtrades = to_async_iterable(eurtrades)
eurgbptrades = to_async_iterable(eurgbptrades)
joint_trade_stream = op.merge_sorted([eurgbptrades,eurtrades], lambda x: x.timestamp)

pub1.topic = 'trades'
joint_trade_stream > pub1